### Connexion de Google Drive

In [13]:
"""from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/journaux-xix-siecle/main
from google.colab.patches import cv2_imshow"""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/journaux-xix-siecle/main


### Téléchargement

In [14]:
!sudo apt install tesseract-ocr
!sudo apt-get install tesseract-ocr-fra
!pip install pytesseract
!pip install Pillow
!pip install opencv-python

!pip install transformers
!pip install transformers[sentencepiece]

!pip install autocorrect

#!git lfs install
#!git clone https://huggingface.co/Jean-Baptiste/camembert-ner

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-fra is already the newest version (4.00~git24-0e00fe6-1.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-p

### Import

In [15]:
import pytesseract
from pytesseract import Output
import cv2
from skimage import io
import numpy as np
import sys
from matplotlib import pyplot as plt
from matplotlib import image as mpimage
from autocorrect import Speller
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import json
import os

### Avoir les noms avec CamemBERT

In [16]:
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [17]:
def camembert(texte):

  res = nlp(texte)

  per = []
  for i in range(len(res)):
    if res[i]:
      if res[i]['entity_group'] == 'PER' and res[i]['score'] >= 0.75:
        per.append(res[i]['word'])

  return per

### Segmentation

In [18]:
def segmentationH(img):

  img = img[10:-10, 10:-10] # remove the border, it confuses contour detection
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  thresholded = cv2.adaptiveThreshold(
                  gray, 255,
                  cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,
                  25,
                  15
              )

  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (500, 1))
  closing = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)

  contours, hierarchy = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  imgc = []
  for i in range(len(contours)):
    x, y, w, h = cv2.boundingRect(contours[i])
    
    if w * h > 11000:
      imgc.append(img[y:y+h, x:x+w])

  imgc.reverse()
  return imgc


def segmentationV(imgc):
  imgd = []
  for i in range(len(imgc)):
    img2 = imgc[i]

    img2 = img2[10:-10, 10:-10] # remove the border, it confuses contour detection

    if(img2.shape[0] > 10 and img2.shape[1] > 10):
      gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)


      thresholded = cv2.adaptiveThreshold(
                      gray, 255,
                      cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,
                      25,
                      15
                  )

      kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 100))
      closing = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)

      contours, hierarchy = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      
      for i in range(len(contours)):
        x, y, w, h = cv2.boundingRect(contours[i])
        
        if w * h > 10000:
          imgd.append(img2[y:y+h, x:x+w])

  imgd.reverse()
  return imgd


def segmentation(img):
  return segmentationV(segmentationH(img))

### OCR-isation

In [19]:
def ocr(img):
  imgString = ''
  imgString += pytesseract.image_to_string(img, lang='fra')
  imgString += '\n'

  imgStringCopy = imgString.replace('-\n', '').replace('\n', ' ').replace('  ','\n')

  return imgStringCopy

### Correction du texte

In [20]:
def correction(texte):

  spell = Speller(lang='fr')
  correction = spell(texte)
    
  return correction

### Main

In [21]:
nomsDict = {}

In [22]:
dir = 'images'
images = []

for image in os.listdir(dir):
  if image.endswith(".jpg") or image.endswith(".tif") or image.endswith(".jpeg") or image.endswith(".png"):
    images.append(image)

In [23]:
for image in images:

  img = cv2.imread(dir + '/' + image)
  segmentationImage = segmentation(img)

  valueInfos = image.split('_')

  for i in range(len(segmentationImage)):
    keyName = image + '_paragraphe_' + str(i+1)

    text = ocr(segmentationImage[i])

    if not text == "":
      print(text)
      try:
        correction = correction(text)
      except:
        correction = text
      noms = camembert(correction)

      if not noms == []:

        d = {
            "nom": noms,
            "date": valueInfos[1],
            "page": valueInfos[3],
            "paragraphe": i+1
        }

        nomsDict[keyName] = d

    else:
      if i != 0:
        i-=1

VU Œ à sucececDocoobüuseovaicaslendasersces LA A —.- Fi Six MOÏS. »esoussaencoegoobsaneostaenecesoe 8 F4 NS ê cr Trois MOÏS cossnecosnocneoosdeneuecuoagesoes 16 LS 7 €?
Le numéro : 45 cenlimes
Le ï DU use tt, fr. . !
A
F Ée : ss
\
. IRTUER ERA EUX Paris, 10, rus des Saints-Pères
On s'abonne à Rome, an bureau de la Civi/{à caftolica via del Gest, 61





























































 Duel
> raç ot epe

 était très-acerédité dans cotte ville que la question romaine allait provoquer des négociations diplomatiques entre les
rançon, ét ils l’on emmené -avec eux dans | leur fuite, ainsi que d'autres employés de | l’admisistration. Îls ont exlorqué environ | 81x cents écus aux caisses publiques et particulièra, et ils ont abattu les insignes ds £ l'autorité pontificale, au milieu de l'indignation ranifests dela population tout entière. À Borghetto, où ils se sont arrêtés dans leur fuite, ils ontcommis des dégâts à la station, en s’emperant de la machine et arrétant 

In [24]:
with open('result/noms.json', 'w') as f:
  json_object = json.dumps(nomsDict, indent=4, ensure_ascii=False)
  f.write(json_object)